In [85]:
import sys
import os

import warnings

import tensorflow as tf
import numpy as np
from tensorflow.data import Dataset
from PIL import Image
from image import *
import tensorflow.image
from keras.optimizers import SGD, Adam
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Conv2D 

import numpy as np
import json
import cv2
import dataset
import time


In [135]:
train_list = []
with open('train_data_list.txt', 'r') as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        train_list.append(inner_list)
train_list = [val.replace('\'','') for val in train_list[0]]

In [136]:
test_list = []
with open('test_data_list.txt', 'r') as f:
    for line in f:
        inner_list = [elt.strip() for elt in line.split(',')]
        test_list.append(inner_list)
test_list = [val.replace('\'','') for val in test_list[0]]

In [176]:
# import image
def load_data(paths, train = True):
    for img_path in paths:
        gt_path = img_path.replace('.jpg','.h5').replace('images','ground-truth')
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels = 3)
        img = tf.image.convert_image_dtype(img, tf.float32)

        gt_file = h5py.File(gt_path)
        target = np.asarray(gt_file['density'])

        target = cv2.resize(target,(int(target.shape[1]/8),int(target.shape[0]/8)),interpolation = cv2.INTER_CUBIC)*64

    
        yield (img, target)

In [187]:
train_dataset = tf.data.Dataset.from_generator(
                        load_data, args = [train_list] , output_types = (tf.float32, tf.float32), output_shapes = ((), (None,)))
train_dataset = train_dataset.shuffle(100000)


In [188]:
test_dataset = tf.data.Dataset.from_generator(
                   load_data, args = [test_list] , output_types = (tf.float32, tf.float32), output_shapes = ((), (None,)))

# Define function

In [60]:
batch_size = 100
epochs = 15
IMG_HEIGHT = 255
IMG_WIDTH = 255

In [61]:
loss_object = tf.keras.losses.MeanSquaredError()

In [64]:
optimizer = Adam(2e-4)

In [91]:
def loss_fn(model, input_image, gt_image):

    output = model(input_image)
    pred = np.sum(output) 
    
    ground_truth = tf.cast(ground_truth, tf.float32)
    ground_truth = np.sum(ground_truth)
    
    # mean squared error
    loss = loss_object(ground_truth, pred)
    return loss.numpy()

In [96]:
## iiterator 사용해야

<DatasetV1Adapter shapes: ((?,), (?, ?)), types: (tf.float32, tf.float32)>

In [109]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(None, None, 3))


In [132]:
for i,(img, target)in enumerate(train_dataset):
        print(target)
        output = model(img)

Tensor("IteratorGetNext_169401:1", shape=(?,), dtype=float32)


ValueError: Input tensor must be of rank 3, 4 or 5 but was 0.